# Query Profiling: mpc_orbits Performance Measurement

Measures query performance against the mpc_orbits table (1.51M rows) to inform
indexing, materialization, and interactive query budget decisions.

Each query runs 3 times; we report median timing and IQR.  EXPLAIN ANALYZE
reveals sequential vs. index scans and buffer hit rates.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lib.db import connect, timed_query, timed_explain, query_log
from lib.orbits import build_orbit_query, build_null_rates_query, build_value_distribution_query

## 1. Define Test Queries

Progressive complexity from unfiltered scan to JSONB extraction.

In [ ]:
# Test query definitions: (label, query_builder_kwargs)
TEST_QUERIES = [
    ("1. Count all",
     dict(count_only=True)),
    
    ("2. Full scan (5 cols)",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"])),
    
    ("3. Filter: orbit_type_int=2 (Apollo)",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"],
          orbit_types=[2])),
    
    ("4. Filter: h > 22",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a", "h"],
          h_range=(22, 35))),
    
    ("5. Filter: NEOs (q < 1.3)",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a", "h"],
          q_range=(0, 1.3))),
    
    ("6. Combined: Apollo + H>22",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a", "h"],
          orbit_types=[2], h_range=(22, 35))),
    
    ("7. NEAs + Tisserand",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"],
          orbit_types=[1, 2, 3, 4], include_tisserand=True)),
    
    ("8. Full scan + Tisserand",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"],
          include_tisserand=True)),
    
    ("9. JSONB: orbit_quality + SNR",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"],
          include_jsonb_fields=["orbit_quality", "snr"])),
    
    ("10. JSONB: all MOIDs",
     dict(columns=["packed_primary_provisional_designation", "earth_moid"],
          include_jsonb_fields=["mars_moid", "venus_moid", "jupiter_moid"])),
]

## 2. Run Each Query 3x, Collect Timings

In [ ]:
N_RUNS = 3
results = []

with connect() as conn:
    for label, kwargs in TEST_QUERIES:
        sql, params = build_orbit_query(**kwargs)
        timings = []
        row_count = 0
        
        for run in range(N_RUNS):
            query_log.clear()
            df = timed_query(conn, sql, params, label=label)
            rec = query_log.records[-1]
            timings.append(rec.elapsed_sec)
            row_count = rec.row_count
            conn.rollback()  # reset transaction state
        
        results.append({
            "label": label,
            "row_count": row_count,
            "median_sec": np.median(timings),
            "q25_sec": np.percentile(timings, 25),
            "q75_sec": np.percentile(timings, 75),
            "min_sec": min(timings),
            "max_sec": max(timings),
            "all_timings": timings,
        })
        print(f"{label}: {row_count:>10,} rows, median {np.median(timings):.3f}s")

timing_df = pd.DataFrame(results)
timing_df

## 3. Query Time Bar Chart

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    y=timing_df["label"],
    x=timing_df["median_sec"],
    orientation="h",
    error_x=dict(
        type="data",
        symmetric=False,
        array=(timing_df["q75_sec"] - timing_df["median_sec"]).tolist(),
        arrayminus=(timing_df["median_sec"] - timing_df["q25_sec"]).tolist(),
    ),
    marker_color="#4363d8",
    hovertemplate="%{y}<br>Median: %{x:.3f}s<br>Rows: %{customdata:,}<extra></extra>",
    customdata=timing_df["row_count"],
))

fig.update_layout(
    title="Query Performance: mpc_orbits (1.51M rows)",
    xaxis_title="Median execution time (seconds)",
    yaxis=dict(autorange="reversed"),
    height=500,
    margin=dict(l=250),
)
fig.show()

## 4. Throughput: Rows vs. Time

In [ ]:
# Exclude count_only queries (row_count = 1)
throughput_df = timing_df[timing_df["row_count"] > 1].copy()
throughput_df["rows_per_sec"] = throughput_df["row_count"] / throughput_df["median_sec"]

fig = px.scatter(
    throughput_df,
    x="row_count",
    y="median_sec",
    text="label",
    hover_data=["rows_per_sec"],
    log_x=True,
    title="Throughput: Row Count vs. Query Time",
    labels={"row_count": "Rows returned", "median_sec": "Median time (s)"},
)
fig.update_traces(textposition="top center", marker_size=10)
fig.show()

## 5. EXPLAIN ANALYZE on Key Queries

In [ ]:
import json

explain_queries = [
    ("Full scan (5 cols)",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"])),
    ("Filter: orbit_type_int=2",
     dict(columns=["packed_primary_provisional_designation", "q", "e", "i", "a"],
          orbit_types=[2])),
    ("JSONB: orbit_quality + SNR",
     dict(columns=["packed_primary_provisional_designation"],
          include_jsonb_fields=["orbit_quality", "snr"])),
]

with connect() as conn:
    for label, kwargs in explain_queries:
        sql, params = build_orbit_query(**kwargs)
        plan = timed_explain(conn, sql, params, label=f"EXPLAIN: {label}")
        
        print(f"\n{'='*60}")
        print(f"EXPLAIN: {label}")
        print(f"{'='*60}")
        
        # Extract key metrics from the plan
        p = plan.get("Plan", plan)
        print(f"  Node Type:     {p.get('Node Type', 'N/A')}")
        print(f"  Actual Rows:   {p.get('Actual Rows', 'N/A'):,}")
        print(f"  Actual Time:   {p.get('Actual Total Time', 'N/A'):.1f} ms")
        print(f"  Shared Hit:    {p.get('Shared Hit Blocks', 'N/A'):,}")
        print(f"  Shared Read:   {p.get('Shared Read Blocks', 'N/A'):,}")
        
        # Full plan for inspection
        print(f"\n  Full plan:")
        print(json.dumps(plan, indent=2)[:2000])

## 6. Timing Variance (Box Plots)

In [ ]:
# Explode all_timings into individual rows for box plot
box_rows = []
for _, row in timing_df.iterrows():
    for t in row["all_timings"]:
        box_rows.append({"label": row["label"], "time_sec": t})
box_df = pd.DataFrame(box_rows)

fig = px.box(
    box_df,
    y="label",
    x="time_sec",
    orientation="h",
    title="Query Timing Variance (3 runs each)",
    labels={"time_sec": "Execution time (seconds)", "label": ""},
)
fig.update_layout(height=500, margin=dict(l=250), yaxis=dict(autorange="reversed"))
fig.show()

## 7. Summary & Recommendations

In [ ]:
print("Query Performance Summary")
print("=" * 60)
query_log.clear()

# Recompute summary from results
for r in results:
    print(f"  {r['label']:<40} {r['row_count']:>10,} rows  {r['median_sec']:>8.3f}s")

print("\nKey Findings:")
full_scan_time = timing_df.loc[timing_df["label"].str.contains("Full scan \\(5"), "median_sec"].values
jsonb_time = timing_df.loc[timing_df["label"].str.contains("JSONB: orbit_quality"), "median_sec"].values

if len(full_scan_time) > 0 and len(jsonb_time) > 0:
    overhead = jsonb_time[0] / full_scan_time[0]
    print(f"  - JSONB extraction overhead: {overhead:.1f}x vs flat column scan")

print("  - See EXPLAIN output above for index vs. sequential scan details")
print("  - Tisserand computation is pure arithmetic — negligible overhead expected")